In [1]:
import json
import pandas as pd
import tensorflow as tf
from datetime import datetime as dt
tf.config.run_functions_eagerly(True)
from hmc.utils.dir import create_dir

2024-06-14 21:25:30.160227: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-14 21:25:30.817612: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from tqdm.notebook import tqdm

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
import os
from datetime import datetime

def horario_atual_para_diretorio():
    # Obtém a data e hora atual
    data_hora_atual = datetime.now()
    
    # Formata a data e hora para o formato de diretório desejado
    diretorio = data_hora_atual.strftime('%Y_%m_%d_%H_%M_%S')
    
    return diretorio

# Exemplo de uso
diretorio_formatado = horario_atual_para_diretorio()

In [5]:
# Set python level verbosity
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.DEBUG)

# Set C++ Graph Execution level verbosity
os.environ['TF_CPP_MIN_LOG_LEVEL'] = str(tf.compat.v1.logging.DEBUG)

base_path = "/mnt/disks/data/fma/trains"
sample_id = "hierarchical_tworoots_dev"


train_path = os.path.join(base_path, sample_id)
tfrecords_path =os.path.join(train_path,'tfrecords')
metadata_path = os.path.join(train_path,"metadata.json")
labels_path = os.path.join(train_path,"labels.json")

model_id = diretorio_formatado

model_path = os.path.join(train_path, model_id)

args = pd.Series({
    "batch_size":32,
    "epochs":10,
    "dropout":0.7,
    'patience':1,
    'max_queue_size':64,
    "labels_path": labels_path,
    "metadata_path": metadata_path,
    "trainset_pattern": os.path.join(tfrecords_path,'train'),
    "testset_pattern": os.path.join(tfrecords_path,'test'),
    "valset_pattern": os.path.join(tfrecords_path,'val'),
    "model_path":model_path
})


In [6]:
create_dir(model_path)

True

In [7]:

import pandas as pd
import tensorflow as tf

import os

BUFFER_SIZE = 10


class Dataset:
    def __init__(self,files,epochs, batch_size, levels_size):
        self.epochs = epochs
        self.batch_size = batch_size
        self.files = files
        self.depth = len(levels_size)
        self.levels_size = levels_size

    def load_dataframe(self, dataset):
        columns = ['features','labels']
        
        
        df = pd.DataFrame(
            dataset.as_numpy_iterator(),
            columns=columns
        )
    
        df.dropna(inplace=True)
    
        return df

    def build(self,df=False):
        
        files = [os.path.join(self.files,file) for file in os.listdir(self.files)]

        ds = tf.data.TFRecordDataset(files)
        
        '''''
            Shuffle and reapeat
        '''''
        
        #ds = ds.shuffle(buffer_size=1024 * 50 * 10)
        #ds = ds.repeat(count=self.epochs)
        
        
        '''''
            Map and batch
        '''''
        
                      
        ds = ds.map(self.__parse__, num_parallel_calls=None)

        if df==True:
            return self.load_dataframe(ds)
        
        ds = ds.batch(self.batch_size, drop_remainder=False)
        
        
                      
        ds = ds.prefetch(buffer_size=5)
        
        
        return ds

    def convert_to_binary(self, label, num_classes):
        # Filtrar índices negativos
        valid_indices = tf.boolean_mask(label, label >= 0)
        
        # Inicializar vetor binário
        binary_label = tf.zeros(num_classes, dtype=tf.float32)
        
        # Atualizar vetor binário com base nos rótulos válidos
        indices = tf.expand_dims(valid_indices, 1)
        updates = tf.ones_like(valid_indices, dtype=tf.float32)
        binary_label = tf.tensor_scatter_nd_update(binary_label, indices, updates)

        return binary_label

    def __parse__(self, element):
        data = {}
        for level in range(1, self.depth+1):
            data[f'label{level}'] = tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True)
        
        data.update({
            'features': tf.io.FixedLenFeature([1280], tf.float32),
            'track_id' : tf.io.FixedLenFeature([], tf.int64),
        })
        
        content = tf.io.parse_single_example(element, data)

        labels = {}
        for level in range(1, self.depth+1):
            local_label = content[f'label{level}']
            binary_label = self.convert_to_binary(local_label, self.levels_size[level-1])
            labels.update({f'level{level}': binary_label})
        

        inp = {"features":content['features'] }

        return inp, labels


In [8]:
'''
from keras import layers
from keras.layers import Dense, Dropout, Reshape, Concatenate, Flatten, Input, Normalization
from keras.optimizers import Adam
import tensorflow as tf

import numpy as np

# class OutputNormalization(layers.Layer):
#     def call(self, x, **kwargs):
#         norm_output = tf.one_hot(tf.math.argmax(x, axis=1), x.shape[1], dtype=x.dtype)
#         norm_output = Flatten()(norm_output)
#         norm_output = Reshape((1,))(norm_output)
#         return norm_output

#     def compute_output_shape(self, input_shape):
#         return input_shape

class OutputNormalization(layers.Layer):
    def call(self, x, **kwargs):
        return tf.one_hot(tf.math.argmax(x, axis=1), x.shape[1], dtype=x.dtype)

    def compute_output_shape(self, input_shape):
        return input_shape



def build_cnn(feature, input_shape):
    x: object = Normalization(input_shape=[input_shape, 1], axis=None)(feature)
    x = layers.Conv1D(128, 3, activation='relu', padding="valid")(x)
    x = layers.MaxPooling1D()(x)
    x = layers.Conv1D(64, 3, activation='relu', padding="valid")(x)
    x = layers.MaxPooling1D()(x)
    x = layers.Conv1D(32, 3, activation='relu', padding="valid")(x)
    x = layers.MaxPooling1D()(x)
    x = layers.Flatten()(x)

    return x


def build_classification(x, size, dropout, input_shape=1024, name='level1'):
    #x: object = Dense(input_shape, activation='relu')(x)
    #x = Dropout(dropout)(x)
    #x = Dense(int(input_shape/2), activation='relu')(x)
    #x = Dropout(dropout)(x)
    x = Dense(int(input_shape/2), activation='relu')(x)
    x = Dropout(dropout)(x)
    x = Dense(size, activation='sigmoid', name=name+'_output')(x)

    return x


def build_model(levels_size: dict, sequence_size: int = 1280, dropout: float = 0.7) -> tf.keras.models.Model:
    """

    :rtype: tf.keras.models.Model
    """
    input_shape = (sequence_size, 1)
    fcn_size = 1024
    input_layer = Input(shape=input_shape, dtype=tf.float32, name="features")
    x = Dense(fcn_size, activation='relu')(input_layer)
    x = Dropout(dropout)(x)
    x = Dense(int(fcn_size / 2), activation='relu')(x)
    x = Dropout(dropout)(x)
    #x_flat = Flatten()(input_layer)
    
    depth = len(levels_size)

    #x: object = build_cnn(music, input_shape)

    # Construção das camadas sequencialmente
    input = x
    outputs = []
    for level, size in levels_size.items():
        # Construir a camada atual
        current_input = input if level == 'level1' else Concatenate(axis=1)([Flatten()(OutputNormalization()(prev_output)), input_layer])
        output = build_classification(current_input, size, dropout,
                                              input_shape=fcn_size, name=level)

        # Atualizar a saída anterior para a próxima iteração
        prev_output = output
        outputs.append(prev_output)
    

    model = tf.keras.models.Model(inputs=input_layer, outputs=outputs, name="Essentia")

    #     _load_weights(model, weights_path)
    
    optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    model.compile(optimizer=optimizer,
                   loss='binary_crossentropy', metrics=['accuracy']*4)

    return model
'''


'\nfrom keras import layers\nfrom keras.layers import Dense, Dropout, Reshape, Concatenate, Flatten, Input, Normalization\nfrom keras.optimizers import Adam\nimport tensorflow as tf\n\nimport numpy as np\n\n# class OutputNormalization(layers.Layer):\n#     def call(self, x, **kwargs):\n#         norm_output = tf.one_hot(tf.math.argmax(x, axis=1), x.shape[1], dtype=x.dtype)\n#         norm_output = Flatten()(norm_output)\n#         norm_output = Reshape((1,))(norm_output)\n#         return norm_output\n\n#     def compute_output_shape(self, input_shape):\n#         return input_shape\n\nclass OutputNormalization(layers.Layer):\n    def call(self, x, **kwargs):\n        return tf.one_hot(tf.math.argmax(x, axis=1), x.shape[1], dtype=x.dtype)\n\n    def compute_output_shape(self, input_shape):\n        return input_shape\n\n\n\ndef build_cnn(feature, input_shape):\n    x: object = Normalization(input_shape=[input_shape, 1], axis=None)(feature)\n    x = layers.Conv1D(128, 3, activation=\'re

In [9]:
import tensorflow as tf
from keras.layers import Layer, Input, Dense, Dropout, Concatenate, BatchNormalization, Flatten, Reshape
from keras.optimizers import Adam
from keras import backend as K
import numpy as np

class OutputNormalization(tf.keras.layers.Layer):
    def call(self, x):
        # Obtemos a classe com a maior probabilidade
        one_hot_encoded = tf.one_hot(tf.math.argmax(x, axis=1), x.shape[1], dtype=x.dtype)
        return one_hot_encoded

    def compute_output_shape(self, input_shape):
        return input_shape

def build_classification(x, size, dropout, input_shape=1024, name='default'):
    x: object = Dense(input_shape, activation='relu')(x)
    x = Dropout(dropout)(x)
    x = Dense(int(input_shape/2), activation='relu')(x)
    x = Dropout(dropout)(x)
    x = Dense(int(input_shape/4), activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout)(x)
    x = Dense(size, activation='softmax', name=name)(x)

    return x


def build_model(levels_size: dict, sequence_size: int = 1280, dropout: float = 0.6) -> tf.keras.models.Model:
    """

    :rtype: tf.keras.models.Model
    """
    input_shape = (sequence_size,)
    music = Input(shape=input_shape, dtype=tf.float32, name="features")
    fcn_size = 1024

    outputs = []
    for level, size in enumerate(levels_size, 1):
        level_name = f'level{str(level)}'
        if level != 1:
            # Aplicar OutputNormalization na saída anterior
            output_normalized = BatchNormalization()(OutputNormalization()(current_output))
            current_input = Concatenate(axis=1)([output_normalized, music])
            current_output = build_classification(current_input, size, dropout, input_shape=fcn_size, name=level_name)
            
        else:
            current_input = music
            current_output = build_classification(current_input, size, dropout, input_shape=fcn_size, name=level_name)

        print(level)
        print(current_output.shape)
        
        # Convert the tensor to a NumPy array
        outputs.append(current_output)
        # Atualizar a saída anterior para a próxima iteração
        current_input = current_output
        


    model = tf.keras.models.Model(inputs=music, outputs=outputs, name="Essentia")

    #     _load_weights(model, weights_path)
    
    #optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']*4, run_eagerly=True)

    return model

In [10]:
with open(args.metadata_path, 'r') as f:
    metadata = json.loads(f.read())
    print(metadata)

{'sequence_size': 1280, 'max_depth': 4, 'levels_size': [2, 29, 15, 2], 'val_path': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/tfrecords/val', 'train_path': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/tfrecords/train', 'test_path': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/tfrecords/test', 'val_torch_path': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/torch/val.pth', 'train_torch_path': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/torch/train.pth', 'test_torch_path': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/torch/test.pth', 'val_csv': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/val.csv', 'train_csv': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/train.csv', 'test_csv': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/test.csv', 'trainset_count': 16791, 'validationset_count': 2007, 'testset_count': 4814}


In [11]:
metadata['trainset_count']

16791

In [12]:
with open(args.labels_path, 'r') as f:
    labels = json.loads(f.read())

In [13]:
labels

{'levels_size': [2, 29, 15, 2],
 'label_1': {'15': 0, '12': 1},
 'label_1_name': {'15': 'Electronic', '12': 'Rock'},
 'label_1_inverse': [15, 12],
 'label_2': {'25': 1,
  '85': 2,
  '45': 3,
  '66': 4,
  '314': 5,
  '70': 6,
  '27': 7,
  '359': 8,
  '297': 9,
  '468': 10,
  '182': 11,
  '181': 12,
  '58': 13,
  '31': 14,
  '495': 15,
  '286': 16,
  '88': 17,
  '26': 18,
  '695': 19,
  '42': 20,
  '236': 21,
  '183': 22,
  '296': 23,
  '184': 24,
  '98': 25,
  '337': 26,
  '440': 27,
  '185': 28,
  '36': 29},
 'label_2_name': {'25': 'Rock>Punk',
  '85': 'Rock>Garage',
  '45': 'Rock>Loud-Rock',
  '66': 'Rock>Indie-Rock',
  '314': 'Rock>Goth',
  '70': 'Rock>Industrial',
  '27': 'Rock>Lo-Fi',
  '359': 'Rock>Shoegaze',
  '297': 'Electronic>Chip Music',
  '468': 'Electronic>Dubstep',
  '182': 'Electronic>House',
  '181': 'Electronic>Techno',
  '58': 'Rock>Psych-Rock',
  '31': 'Rock>Metal',
  '495': 'Electronic>Downtempo',
  '286': 'Electronic>Trip-Hop',
  '88': 'Rock>New Wave',
  '26': 'Rock

In [14]:
for level in labels['levels_size']:
    print(level)

2
29
15
2


In [15]:
depth = len(labels['levels_size'])

In [16]:
depth

4

In [17]:
#ds_train = Dataset(args.trainset_pattern, args.epochs, args.batch_size, levels_size).build(df=False)

In [18]:
#df_train = Dataset(args.trainset_pattern, args.epochs, args.batch_size, levels_size).build(df=True)

In [19]:
from tensorflow.keras.callbacks import EarlyStopping
# from sabotage.model.callback import ValidateCallback, BackupAndRestoreCheckpoint

def run(args: object):
    print(args)

    with open(args.metadata_path, 'r') as f:
        metadata = json.loads(f.read())
        print(metadata)

    with open(args.labels_path, 'r') as f:
        labels = json.loads(f.read())

    params: dict = {
        'levels_size': labels['levels_size'],
        'sequence_size': metadata['sequence_size'],
        'dropout': args.dropout
    }

    print(params['levels_size'])
    model = build_model(**params)
    print(model.summary())
    
    tf.keras.utils.plot_model(
        model,
        to_file="model.png",
        show_shapes=False,
        show_dtype=False,
        show_layer_names=True,
        rankdir="TB",
        expand_nested=False,
        dpi=96,
        layer_range=None,
        show_layer_activations=False,
        show_trainable=False,
    )
    
    ds_train = Dataset(args.trainset_pattern, args.epochs, args.batch_size, params['levels_size']).build(df=False)
    ds_validation = Dataset(args.valset_pattern, args.epochs, args.batch_size, params['levels_size']).build(df=False)
    callbacks = [EarlyStopping(monitor='loss', patience=args.patience, verbose=1)]
    model.fit(ds_train,
              validation_data=ds_validation,
              steps_per_epoch=metadata['trainset_count'] // args.batch_size,
              validation_steps=metadata['validationset_count'] // args.batch_size,
              epochs=args.epochs,
              callbacks=callbacks)

    model.save(os.path.join(args.model_path, 'best_binary.keras'))


In [20]:

time_start = dt.utcnow()
print("[{}] Experiment started at {}".format(id, time_start.strftime("%H:%M:%S")))
print(".......................................")
print(args)
run(args)
time_end = dt.utcnow()
time_elapsed = time_end - time_start
print(".......................................")
print("[{}] Experiment finished at {} / elapsed time {}s".format(id, time_end.strftime("%H:%M:%S"), time_elapsed.total_seconds()))

[<built-in function id>] Experiment started at 00:25:32
.......................................
batch_size                                                         32
epochs                                                             10
dropout                                                           0.7
patience                                                            1
max_queue_size                                                     64
labels_path         /mnt/disks/data/fma/trains/hierarchical_tworoo...
metadata_path       /mnt/disks/data/fma/trains/hierarchical_tworoo...
trainset_pattern    /mnt/disks/data/fma/trains/hierarchical_tworoo...
testset_pattern     /mnt/disks/data/fma/trains/hierarchical_tworoo...
valset_pattern      /mnt/disks/data/fma/trains/hierarchical_tworoo...
model_path          /mnt/disks/data/fma/trains/hierarchical_tworoo...
dtype: object
batch_size                                                         32
epochs                                            

2024-06-14 21:25:32.876426: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9786 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


Model: "Essentia"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ features            │ (None, 1280)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1024)      │  1,311,744 │ features[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 1024)      │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 512)       │    524,800 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 512)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256)       │    131,328 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256)       │      1,024 │ dense_2[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 256)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ level1 (Dense)      │ (None, 2)         │        514 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_normalizati… │ (None, 2)         │          0 │ level1[0][0]      │
│ (OutputNormalizati… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 2)         │          8 │ output_normaliza… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 1282)      │          0 │ batch_normalizat… │
│ (Concatenate)       │                   │            │ features[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1024)      │  1,313,792 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 1024)      │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 512)       │    524,800 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 512)       │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 256)       │    131,328 │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256)       │      1,024 │ dense_5[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 256)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ level2 (Dense)      │ (None, 29)        │      7,453 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_normalizati… │ (None, 29)        │          0 │ level2[0][0]      │
│ (OutputNormalizati… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 7,935,208 (30.27 MB)

...

Epoch 1/10


2024-06-14 21:25:33.742882: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at scatter_nd_op.cc:218 : INVALID_ARGUMENT: indices[0] = [2] does not index into shape [2]
2024-06-14 21:25:33.749038: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at scatter_nd_op.cc:218 : INVALID_ARGUMENT: indices[0] = [15] does not index into shape [15]


  2/524 ━━━━━━━━━━━━━━━━━━━━ 2:08 246ms/step - level1_accuracy: 0.4766 - level2_accuracy: 0.0312 - level3_accuracy: 0.0938 - level4_accuracy: 0.4375 - loss: 4.8421

2024-06-14 21:25:36.135701: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: indices[0] = [2] does not index into shape [2]
	 [[{{node TensorScatterUpdate_3}}]]
2024-06-14 21:25:36.136483: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at scatter_nd_op.cc:218 : INVALID_ARGUMENT: indices[0] = [15] does not index into shape [15]


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_5_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[0] = [2] does not index into shape [2]
	 [[{{node TensorScatterUpdate_3}}]] [Op:IteratorGetNext] name: 